In [1]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import torch
import utils
import test

from model import generate_model
import torch.nn as nn

import dataloader
import pandas as pd
import os
from torch.utils.data import DataLoader

In [2]:
config = utils.load_config()

In [3]:
#데이터셋 분리(Train, validation, test)
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
df_dataset = df_dataset.dropna().reset_index(drop=True)
df_oasis = df_dataset[df_dataset['source'] == 'OASIS-3']
df_adni = df_dataset[df_dataset['source'] == 'ADNI']
X_train,X_val,y_train,y_val = dataloader.dataset_split(df_oasis)
X_test = df_adni.drop(labels='group_maxinc',axis=1)
y_test = df_adni['group_maxinc']

traindata=dataloader.MRIDataset(X_train,y_train)
valdata=dataloader.MRIDataset(X_val,y_val)
testdata=dataloader.MRIDataset(X_test,y_test)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(traindata, batch_size=1, shuffle=True,num_workers=1)
val_dataloader  = DataLoader(valdata, batch_size=1, shuffle=False,num_workers=1)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

print('train_dataloader : ',len(train_dataloader.dataset))
print('val_dataloader : ',len(val_dataloader.dataset))
print('test_dataloader : ',len(test_dataloader.dataset))

train_dataloader :  659
val_dataloader :  165
test_dataloader :  166


d:\MyGit\BrainMR_MCI\dataloader.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['grp'] = (df_dataset['source'].str.replace('OASIS-3','1').str.replace('ADNI','2').apply(pd.to_numeric)*1000


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load('result/resnet50_best.pth', map_location=device)

In [7]:
model, _ = generate_model(model_name='resnet',model_depth = 50,n_classes=3,resnet_shortcut='B')
model.to(device)
model.load_state_dict(checkpoint['state_dict'])

log_path = config['log_path']
criterion_clf = nn.CrossEntropyLoss().to(device)


In [8]:
import numpy as np
outputs = []
labels = []
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_dataloader):
        output = model(inputs)
        outputs.append(output[0].data.cpu())
        labels.append(targets[0].data.cpu())
        
outputs = torch.stack(outputs)
labels =  torch.stack(labels)

In [19]:
val_outputs = []
val_labels = []
with torch.no_grad():
    for i, (inputs, targets) in enumerate(val_dataloader):
        output = model(inputs)
        val_outputs.append(output[0].data.cpu())
        val_labels.append(targets[0].data.cpu())
        
val_outputs = torch.stack(val_outputs)
val_labels =  torch.stack(val_labels)

In [30]:
val_outputs.argmax()

tensor(207)

In [14]:
from torchmetrics.functional.classification import multiclass_auroc
multiclass_auroc(outputs, labels, num_classes=3, average="macro", thresholds=None)

tensor(0.5452)

In [26]:
multiclass_auroc(val_outputs, val_labels, num_classes=3, average="macro", thresholds=None)

tensor(0.7389)

In [15]:
from torchmetrics.classification import MulticlassConfusionMatrix
metric = MulticlassConfusionMatrix(num_classes=3)
metric(outputs, labels)

tensor([[52,  0,  0],
        [80,  0,  0],
        [34,  0,  0]])

In [20]:
metric(val_outputs, val_labels)

tensor([[108,   0,   0],
        [ 25,   0,   0],
        [ 32,   0,   0]])

In [31]:
val_labels

tensor([1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        2, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0,
        0, 2, 2, 1, 0, 2, 0, 0, 0, 2, 1, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2, 1, 0,
        2, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0])

In [22]:
val_outputs

tensor([[ 2.8220, -0.0150, -3.6971],
        [ 3.4045, -0.3355, -4.0506],
        [ 5.1177, -1.3357, -4.8616],
        [ 4.9494, -1.3222, -4.6703],
        [ 1.3264,  0.5941, -2.7710],
        [ 3.5377, -0.3451, -4.1849],
        [ 3.7945, -0.5872, -4.2347],
        [ 2.5396,  0.0548, -3.4791],
        [ 2.4038,  0.6729, -3.8973],
        [ 5.3875, -1.5676, -4.9099],
        [ 1.2746, -0.2852, -1.9068],
        [ 4.4810, -0.9907, -4.4974],
        [ 1.3528, -0.3609, -1.9378],
        [ 5.2654, -1.9413, -4.4077],
        [ 2.3668,  0.4183, -3.7340],
        [ 2.7784,  0.2527, -3.9285],
        [ 4.6914, -1.1050, -4.5934],
        [ 5.1774, -1.4485, -4.7772],
        [ 5.3011, -1.5726, -4.7789],
        [ 1.9441,  0.6298, -3.4114],
        [ 4.9028, -1.3319, -4.6519],
        [ 4.0084, -0.7924, -4.2314],
        [ 2.5584,  0.4325, -3.8026],
        [ 2.9758,  0.0798, -3.9548],
        [ 4.5163, -0.8023, -4.7684],
        [ 3.5012, -0.3040, -4.1709],
        [ 1.6743,  0.3433, -2.8638],
 

In [34]:
from torchmetrics.classification import MulticlassConfusionMatrix
target = torch.tensor([2, 1, 0, 0])
preds = torch.tensor([[0.96, 0.26, 0.58],
                      [0.92, 0.61, 0.17],
                      [0.91, 0.09, 0.20],
                      [0.95, 0.82, 0.13],
])
metric = MulticlassConfusionMatrix(num_classes=3)
metric(preds, target)

tensor([[2, 0, 0],
        [1, 0, 0],
        [1, 0, 0]])